In [1]:
import pandas as pd
import os
import ast

El código que has proporcionado se utiliza para leer un archivo JSON llamado 'australian_users_items.json' y cargar su contenido en un DataFrame de Pandas. Sin embargo, en lugar de usar la función json.loads de la biblioteca estándar de Python o la función pandas.read_json, utiliza la biblioteca ast para evaluar cada línea del archivo como una expresión de Python y luego lo almacena en un DataFrame. Veamos el código detalladamente:

Explicación paso a paso:

Importa las bibliotecas ast y pandas. ast se utiliza para evaluar expresiones de Python, y pandas es la biblioteca de análisis de datos que se utilizará para crear el DataFrame.

Crea una lista vacía llamada rows_items que se utilizará para almacenar las filas (diccionarios) de datos.

Abre el archivo 'australian_users_items.json' en modo de lectura ('r') y especifica la codificación como UTF-8.

Itera a través de las líneas del archivo utilizando un bucle for. items.readlines() devuelve una lista de todas las líneas en el archivo.

En cada iteración del bucle, la línea actual se evalúa como una expresión de Python utilizando ast.literal_eval(line). Esto se hace para convertir cada línea del archivo en un diccionario Python válido. Luego, ese diccionario se agrega a la lista rows_items.

Finalmente, se crea un DataFrame de Pandas llamado users_items utilizando pd.DataFrame(rows_items). El DataFrame se rellena con los diccionarios convertidos a partir del archivo JSON.

Este código se utiliza cuando el archivo JSON no está formateado de manera convencional, por lo que no se puede cargar directamente con pandas.read_json o json.loads. En lugar de eso, utiliza ast.literal_eval para evaluar cada línea del archivo como código Python y, posteriormente, almacena los resultados en un DataFrame de Pandas.

In [2]:
# Crea una lista vacía para almacenar las filas (diccionarios) de datos
rows_items = []
# Abre el archivo 'australian_users_items.json' para lectura
with open('../datasets/australian_users_items.json', 'r', encoding='UTF-8') as items:
    # Itera a través de las líneas del archivo
    for line in items.readlines():
        # Evalúa cada línea como una expresión de Python y agrega el resultado a la lista rows_items
        rows_items.append(ast.literal_eval(line))

# Crea un DataFrame de Pandas a partir de las filas (diccionarios) de datos
users_items = pd.DataFrame(rows_items)

# Elimina columna que no se considera necesaria
users_items = users_items.drop(columns=['user_url'])

users_items.head(3)

,user_id,items_count,steam_id,items
0,76561197970982479,277,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [3]:
# Crea una lista vacía para almacenar los datos desanidados
des_items = []

# Itera las filas del DataFrame 'users_items'
for index, row in users_items.iterrows():
    # Obtiene los valores de las columnas 'user_id', 'steam_id' y 'items' de la fila actual
    user_id = row['user_id']
    steam_id = row['steam_id']
    items = row['items']

    # Itera a través de la lista de ítems en la columna 'items'
    for i in items:
        # Crea un diccionario con los valores desanidados
        items_desanidados = {
            'user_id': user_id,
            'steam_id': steam_id,
            'item_id': i.get('item_id', ''), # Obtiene el valor de 'item_id'
            'item_name': i.get('item_name',''), # Obtener el valor de 'item_name'
            'playtime_forever': i.get('playtime_forever',''), # Obtener el valor de 'playtime_forever'
            'playtime_2weeks': i.get('playtime_2weeks','') # Obtener el valor de 'playtime_2weeks'
        }
        # Agrega el diccionario a la lista des_items
        des_items.append(items_desanidados)

# Crea un nuevo DataFrame a partir de la lista des_items
users_items = pd.DataFrame(des_items)

users_items.head(3)

,user_id,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,76561197970982479,30,Day of Defeat,7,0


In [4]:
# Elimina la columna 'steam_id'
users_items = users_items.drop(columns='steam_id')

In [5]:
# Filtra el DataFrame users_items para mantener solo las filas donde 'playtime_forever' no sea igual a 0
users_items = users_items[users_items['playtime_forever'] != 0]

In [7]:
# Guarda el DataFrame steam_games en un archivo Parquet
users_items.to_parquet(os.path.join('../datasets/', 'users_items.parquet'), index=False)